In [1]:
import logging
LOG_FORMAT = "%(asctime)s - %(levelname)s - %(message)s"    # 日志格式化输出
DATE_FORMAT = "%m/%d/%Y %H:%M:%S %p"                        # 日期格式
fp = logging.FileHandler('esio.log')
fs = logging.StreamHandler()
logging.basicConfig(level=logging.WARNING, format=LOG_FORMAT, datefmt=DATE_FORMAT, handlers=[fs])    # 调用

import pandas as pd
from pandasticsearch import DataFrame, Select
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from tqdm import tqdm

from sklearn import datasets

In [2]:
df = datasets.load_iris()

In [3]:
df = pd.DataFrame(df.data, columns=['x1','x2','x3','x4'])
df

,x1,x2,x3,x4
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [4]:
# logging.debug("This is a debug log.")
# logging.info("This is a info log.")
# logging.warning("This is a warning log.")
# logging.error("This is a error log.")
# logging.critical("This is a critical log.")

In [5]:
es_localhost = "192.168.3.193:9200"
es_ysyp_test = "115.29.34.243:9200"
unsername = 'elastic'
password = 'superzsh123'

es = Elasticsearch(
    [es_ysyp_test],
    http_auth=(unsername, password)
)

In [6]:
# doc = {
#     'item':'1038111833',
#     'sim':['1098181833','1038131861'],
#     'sf':0.6
# }
# res = es.index(index="index_recall_source", doc_type='type_recall_source', body=doc)
# es.indices.refresh(index="index_recall_source")

In [7]:
# %%time
# # df = DataFrame.from_es(url='http://115.29.34.243:9200', username=unsername, password=password, index='index_recall_source', doc_type='type_recall_source', compat=5).to_pandas()
# # df = DataFrame.from_es(url='http://'+es_localhost, index='index_recall_pool', doc_type='type_recall_u2i', compat=7).to_pandas()
# result_dict = es.search(index="index_recall_source", body={"query": {"match_all": {}}})
# df = Select.from_dict(result_dict).to_pandas()
# df.drop(['_index', '_type', '_id', '_score'], axis=1, inplace=True)

In [8]:
# for i in range(0, 49):
#     df =df.append(df.loc[[0]],ignore_index=True)
# df

In [9]:
%%time
batch_size = 10
es_index = "index_recall_pool"
es_type = "type_recall_i2i"

for i in tqdm(range(0, len(df), batch_size)):
    tmp = df[i:i+batch_size]
    ACTIONS = []
    tmp.apply(lambda x:ACTIONS.append(
        {"_index": es_index,
        "_type": es_type,
        "_source":eval(x.to_json(orient='index'))}),axis=1)   
    success, _ =  bulk(es, ACTIONS, index="index_recall_pool", raise_on_error=True)
    es.indices.refresh(index="index_recall_pool")
    print(tmp)

 20%|██        | 3/15 [00:00<00:01, 10.32it/s]

    x1   x2   x3   x4
0  5.1  3.5  1.4  0.2
1  4.9  3.0  1.4  0.2
2  4.7  3.2  1.3  0.2
3  4.6  3.1  1.5  0.2
4  5.0  3.6  1.4  0.2
5  5.4  3.9  1.7  0.4
6  4.6  3.4  1.4  0.3
7  5.0  3.4  1.5  0.2
8  4.4  2.9  1.4  0.2
9  4.9  3.1  1.5  0.1
     x1   x2   x3   x4
10  5.4  3.7  1.5  0.2
11  4.8  3.4  1.6  0.2
12  4.8  3.0  1.4  0.1
13  4.3  3.0  1.1  0.1
14  5.8  4.0  1.2  0.2
15  5.7  4.4  1.5  0.4
16  5.4  3.9  1.3  0.4
17  5.1  3.5  1.4  0.3
18  5.7  3.8  1.7  0.3
19  5.1  3.8  1.5  0.3
     x1   x2   x3   x4
20  5.4  3.4  1.7  0.2
21  5.1  3.7  1.5  0.4
22  4.6  3.6  1.0  0.2
23  5.1  3.3  1.7  0.5
24  4.8  3.4  1.9  0.2
25  5.0  3.0  1.6  0.2
26  5.0  3.4  1.6  0.4
27  5.2  3.5  1.5  0.2
28  5.2  3.4  1.4  0.2
29  4.7  3.2  1.6  0.2


 33%|███▎      | 5/15 [00:00<00:00, 10.24it/s]

     x1   x2   x3   x4
30  4.8  3.1  1.6  0.2
31  5.4  3.4  1.5  0.4
32  5.2  4.1  1.5  0.1
33  5.5  4.2  1.4  0.2
34  4.9  3.1  1.5  0.2
35  5.0  3.2  1.2  0.2
36  5.5  3.5  1.3  0.2
37  4.9  3.6  1.4  0.1
38  4.4  3.0  1.3  0.2
39  5.1  3.4  1.5  0.2
     x1   x2   x3   x4
40  5.0  3.5  1.3  0.3
41  4.5  2.3  1.3  0.3
42  4.4  3.2  1.3  0.2
43  5.0  3.5  1.6  0.6
44  5.1  3.8  1.9  0.4
45  4.8  3.0  1.4  0.3
46  5.1  3.8  1.6  0.2
47  4.6  3.2  1.4  0.2
48  5.3  3.7  1.5  0.2
49  5.0  3.3  1.4  0.2


 47%|████▋     | 7/15 [00:00<00:00, 10.56it/s]

...

     x1   x2   x3   x4
80  5.5  2.4  3.8  1.1
81  5.5  2.4  3.7  1.0
82  5.8  2.7  3.9  1.2
83  6.0  2.7  5.1  1.6
84  5.4  3.0  4.5  1.5
85  6.0  3.4  4.5  1.6
86  6.7  3.1  4.7  1.5
87  6.3  2.3  4.4  1.3
88  5.6  3.0  4.1  1.3
89  5.5  2.5  4.0  1.3
     x1   x2   x3   x4
90  5.5  2.6  4.4  1.2
91  6.1  3.0  4.6  1.4
92  5.8  2.6  4.0  1.2
93  5.0  2.3  3.3  1.0
94  5.6  2.7  4.2  1.3
95  5.7  3.0  4.2  1.2
96  5.7  2.9  4.2  1.3
97  6.2  2.9  4.3  1.3
98  5.1  2.5  3.0  1.1
99  5.7  2.8  4.1  1.3
      x1   x2   x3   x4
100  6.3  3.3  6.0  2.5
101  5.8  2.7  5.1  1.9
102  7.1  3.0  5.9  2.1
103  6.3  2.9  5.6  1.8
104  6.5  3.0  5.8  2.2
105  7.6  3.0  6.6  2.1
106  4.9  2.5  4.5  1.7
107  7.3  2.9  6.3  1.8
108  6.7  2.5  5.8  1.8
109  7.2  3.6  6.1  2.5


 87%|████████▋ | 13/15 [00:01<00:00, 10.64it/s]

      x1   x2   x3   x4
110  6.5  3.2  5.1  2.0
111  6.4  2.7  5.3  1.9
112  6.8  3.0  5.5  2.1
113  5.7  2.5  5.0  2.0
114  5.8  2.8  5.1  2.4
115  6.4  3.2  5.3  2.3
116  6.5  3.0  5.5  1.8
117  7.7  3.8  6.7  2.2
118  7.7  2.6  6.9  2.3
119  6.0  2.2  5.0  1.5
      x1   x2   x3   x4
120  6.9  3.2  5.7  2.3
121  5.6  2.8  4.9  2.0
122  7.7  2.8  6.7  2.0
123  6.3  2.7  4.9  1.8
124  6.7  3.3  5.7  2.1
125  7.2  3.2  6.0  1.8
126  6.2  2.8  4.8  1.8
127  6.1  3.0  4.9  1.8
128  6.4  2.8  5.6  2.1
129  7.2  3.0  5.8  1.6
      x1   x2   x3   x4
130  7.4  2.8  6.1  1.9
131  7.9  3.8  6.4  2.0
132  6.4  2.8  5.6  2.2
133  6.3  2.8  5.1  1.5
134  6.1  2.6  5.6  1.4
135  7.7  3.0  6.1  2.3
136  6.3  3.4  5.6  2.4
137  6.4  3.1  5.5  1.8
138  6.0  3.0  4.8  1.8
139  6.9  3.1  5.4  2.1


100%|██████████| 15/15 [00:01<00:00, 10.92it/s]

      x1   x2   x3   x4
140  6.7  3.1  5.6  2.4
141  6.9  3.1  5.1  2.3
142  5.8  2.7  5.1  1.9
143  6.8  3.2  5.9  2.3
144  6.7  3.3  5.7  2.5
145  6.7  3.0  5.2  2.3
146  6.3  2.5  5.0  1.9
147  6.5  3.0  5.2  2.0
148  6.2  3.4  5.4  2.3
149  5.9  3.0  5.1  1.8
CPU times: user 297 ms, sys: 16.4 ms, total: 314 ms
Wall time: 1.38 s


In [10]:
%%time
# df = DataFrame.from_es(url='http://115.29.34.243:9200', username=unsername, password=password, index='index_recall_pool', doc_type='type_recall_i2i', compat=5).to_pandas()
result_dict = es.search(index="index_recall_pool", body={"query": {"match_all": {}}})
df = Select.from_dict(result_dict).to_pandas()
df

CPU times: user 7.18 ms, sys: 107 µs, total: 7.28 ms
Wall time: 25.8 ms


,_index,_type,_id,_score,x1,x2,x3,x4
0,index_recall_pool,type_recall_i2i,AW613qWEBFL3W0xEyFEW,1.0,5.4,3.9,1.7,0.4
1,index_recall_pool,type_recall_i2i,AW613qWEBFL3W0xEyFEX,1.0,4.6,3.4,1.4,0.3
2,index_recall_pool,type_recall_i2i,AW613qWEBFL3W0xEyFEd,1.0,4.8,3.0,1.4,0.1
3,index_recall_pool,type_recall_i2i,AW613qWEBFL3W0xEyFEe,1.0,4.3,3.0,1.1,0.1
4,index_recall_pool,type_recall_i2i,AW613qWEBFL3W0xEyFEi,1.0,5.1,3.5,1.4,0.3
5,index_recall_pool,type_recall_i2i,AW613qWEBFL3W0xEyFEl,1.0,5.4,3.4,1.7,0.2
6,index_recall_pool,type_recall_i2i,AW613qWEBFL3W0xEyFEm,1.0,5.1,3.7,1.5,0.4
7,index_recall_pool,type_recall_i2i,AW613qWEBFL3W0xEyFEr,1.0,5.0,3.4,1.6,0.4
8,index_recall_pool,type_recall_i2i,AW613qWEBFL3W0xEyFEs,1.0,5.2,3.5,1.5,0.2
9,index_recall_pool,type_recall_i2i,AW613qWEBFL3W0xEyFEw,1.0,5.4,3.4,1.5,0.4


In [11]:
# es.indices.delete(index='index_recall_source')
# es.indices.delete(index='index_recall_pool')